# Este entrenamiento prueba el algoritmo vpg

### Instalar mujoco

In [ ]:
#Include this at the top of your colab code
import os
if not os.path.exists('.mujoco_setup_complete'):
  # Get the prereqs
  !apt-get -qq update
  !apt-get -qq install -y libosmesa6-dev libgl1-mesa-glx libglfw3 libgl1-mesa-dev libglew-dev patchelf
  # Get Mujoco
  !mkdir ~/.mujoco
  !wget -q https://mujoco.org/download/mujoco210-linux-x86_64.tar.gz -O mujoco.tar.gz
  !tar -zxf mujoco.tar.gz -C "$HOME/.mujoco"
  !rm mujoco.tar.gz
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  !echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$HOME/.mujoco/mujoco210/bin' >> ~/.bashrc 
  !echo 'export LD_PRELOAD=$LD_PRELOAD:/usr/lib/x86_64-linux-gnu/libGLEW.so' >> ~/.bashrc 
  # THE ANNOYING ONE, FORCE IT INTO LDCONFIG SO WE ACTUALLY GET ACCESS TO IT THIS SESSION
  !echo "/root/.mujoco/mujoco210/bin" > /etc/ld.so.conf.d/mujoco_ld_lib_path.conf
  !ldconfig
  # Install Mujoco-py
  !pip3 install -U 'mujoco-py<2.2,>=2.1'
  # run once
  !touch .mujoco_setup_complete

try:
  if _mujoco_run_once:
    pass
except NameError:
  _mujoco_run_once = False
if not _mujoco_run_once:
  # Add it to the actively loaded path and the bashrc path (these only do so much)
  try:
    os.environ['LD_LIBRARY_PATH']=os.environ['LD_LIBRARY_PATH'] + ':/root/.mujoco/mujoco210/bin'
  except KeyError:
    os.environ['LD_LIBRARY_PATH']='/root/.mujoco/mujoco210/bin'
  try:
    os.environ['LD_PRELOAD']=os.environ['LD_PRELOAD'] + ':/usr/lib/x86_64-linux-gnu/libGLEW.so'
  except KeyError:
    os.environ['LD_PRELOAD']='/usr/lib/x86_64-linux-gnu/libGLEW.so'
  # presetup so we don't see output on first env initialization
  import mujoco_py
  _mujoco_run_once = True


### instalar librerias

In [ ]:
#clonar TEG 
#!rm -r TEG/ esta funcion se usa en caso de tener que remover el proyecto
!git clone https://github.com/Alexfm101/TEG.git
!pip3 install -e TEG

In [ ]:
#clonar spinning up
!sudo apt-get update && sudo apt-get install libopenmpi-dev
!git clone https://github.com/openai/spinningup.git
!pip3 install -e spinningup

### configurar librerias para servidor(opcional)

In [ ]:
# IMPORTAR LA LIBRERIA AL SYS
import sys
sys.path.append('TEG') #para usar sin el TEG en el import

### importar  librerias (probar desde aqui)

In [ ]:
import gym
import numpy as np
from RobotEnv.envs.UR5_Env import UR5_EnvTest #acuerdate de agregar TEG para colab
from spinningup.spinup import vpg_pytorch as vpg
import spinningup.spinup.algos.pytorch.vpg.core as core

### configurar entrenamiento



In [ ]:
# entrenar
env = lambda: UR5_EnvTest(simulation_frames=100, torque_control= 0.01, distance_threshold=0.5, Gui=False)

logger_kwargs = dict(output_dir='TEG/data_exp1', exp_name='robot_train')

vpg(env, actor_critic=core.MLPActorCritic, ac_kwargs=dict(), seed=0,
    steps_per_epoch=4000, epochs=50, gamma=0.99, pi_lr=3e-4,
    vf_lr=1e-3, train_v_iters=80,lam=0.97, max_ep_len=1000,
    logger_kwargs=logger_kwargs, save_freq=10)